In [1]:
import numpy as np
import cv2
import os
from numpy import genfromtxt
import json
import math

# FPS of the generated frames by manip.m script (accumulation_dt=50ms --> FPS=20)
ANNOT_FPS = 20

# annotated line color and thickness
color = 255
thickness = 2

root_dir = '~/ev_snn_percept/binary/sample_pavement/'

In [35]:
# convert matlab generated csvs to png files and store them in binary/sample_pavement/png/ directory
for filename in sorted(os.listdir(root_dir + 'csv/')):
    img_np = genfromtxt( root_dir + 'csv/' + filename, delimiter=',')
    img_np[img_np ==  1] = 255 # on events
    img_np[img_np == -1] = 1   # off events
    img_np[img_np ==  0] = 127 # background

    cv2.imwrite(root_dir + '/png/' + filename.replace('.csv', '.png'), img_np)

In [36]:
# create labelme annotations and save them in binary/sample_pavement/json/ directory
LABELME_AFTER_FRAME = 0
for filename in sorted(os.listdir(root_dir + 'png/')):
    file_cnt = int(filename.replace('.png', ''))
    if file_cnt >= LABELME_AFTER_FRAME:
        if (not os.path.isfile(root_dir + 'json/' + filename.replace('.png', '.json'))):
            os.system('labelme ' + root_dir + 'png/' + filename + ' -O ' + root_dir + 'json/' + filename.replace('.png', '.json'))

In [37]:
# generate r,theta (annotated) from json files, save them in numpy format and visualize
annot_list = []

for filename in sorted(os.listdir(root_dir + 'json/')):
    
    # first retrieve information from json files
    file_cnt = int(filename.replace('.json', ''))
    
    f = open(root_dir + 'json/' + filename)
    data = json.load(f)
    f.close()

    data_dict = data.get('shapes')

    first_point = data_dict[0]['points'][0]
    second_point = data_dict[0]['points'][1]

    # convert line represented by two x-y plane points to one r-theta polar plane point
    # r expressed in 128 pixel range
    # theta expressed in radians
    if second_point[0] == first_point[0]:
        theta = 0
        r = second_point[0]
    elif second_point[1] == first_point[1]:
        theta = np.pi/2
        r = second_point[1]
    else:
        theta = math.atan( -(first_point[0] - second_point[0]) / (first_point[1] - second_point[1]) )   
        r = first_point[0]*math.cos(theta) + first_point[1]*math.sin(theta)
    
    # save annotations in numpy format 
    # column 1: r, column 2: theta, column 3: timestamp (msec)
    annot_list.append((r, theta, (file_cnt+1)*(1000/ANNOT_FPS)))
    np.save(root_dir + 'annot_pav.npy', annot_list)

    # visualize
    img = cv2.imread(root_dir + 'png/' + filename.replace('.json', '.png'))
    start_point = (round(first_point[0]), round(first_point[1]))
    end_point = (round(second_point[0]), round(second_point[1]))

    img = cv2.line(img, start_point, end_point, color, thickness)
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA)
    cv2.imwrite(root_dir + 'vis_annot/' + filename.replace('.json', '.png') , img)